In [7]:
#Import
import geopandas as gpd
import pandas as pd
import numpy as np

#Read data
svi_data = pd.read_csv("./CAL_SVI.csv")
demographic_data=pd.read_csv("./Demographic_CAL.csv")
esg_data= gpd.read_file('./metrics_06.geojson')


#Read heads
print(svi_data.head())
print(demographic_data.head())
print(esg_data.head())

        GEOID   Total
0  6001400200  0.1494
1  6001400300  0.2771
2  6001400400  0.2121
3  6001400500  0.1184
4  6001400600  0.2059
        GEOID  PercU18  Total  Perc_two_or_more  Perc_white  Perc_black_afam  \
0  6001400100     15.6   3120               6.0        76.7              3.4   
1  6001400200     18.4   2007               9.3        78.6              2.6   
2  6001400300     15.6   5051               1.9        74.2              9.1   
3  6001400400     18.6   4007               7.6        68.6              7.3   
4  6001400500     14.5   4124              15.2        49.4             21.4   

   Perc_amindian_alnative  Perc_asian  Perc_hawaii_pi  Perc_other_race  
0                     0.0        12.8             0.0              1.1  
1                     0.2         8.5             0.0              0.8  
2                     0.0        11.7             0.5              2.7  
3                     1.1        10.5             0.0              5.0  
4                     

In [8]:
svi_data.rename({'Total': 'SVI_total'}, axis=1, inplace=True)
print(svi_data.dtypes)
print(demographic_data.dtypes)

GEOID          int64
SVI_total    float64
dtype: object
GEOID                       int64
PercU18                   float64
Total                       int64
Perc_two_or_more          float64
Perc_white                float64
Perc_black_afam           float64
Perc_amindian_alnative    float64
Perc_asian                float64
Perc_hawaii_pi            float64
Perc_other_race           float64
dtype: object


In [9]:
#Drop last three digits of the ESG data GEOID
esg_data['GEOID']=esg_data['GEOID'].apply(str)
print(esg_data.dtypes)
esg_data['GEOID']=esg_data['GEOID'].str.slice(1, 11)
print(esg_data.head())


GEOID                         object
jobs_access_pct              float64
education_diversity_pct      float64
hospitality_access_pct       float64
live_work_score_pct          float64
shopping_access_pct          float64
income_diversity_pct         float64
employment_diversity_pct     float64
income_harmony_pct           float64
edu_harmony_pct              float64
healthcare_access_pct        float64
geometry                    geometry
dtype: object
        GEOID  jobs_access_pct  education_diversity_pct  \
0  6037800408         0.830531                 0.202880   
1  6041124200         0.578893                 0.598638   
2  6059062619         0.864071                 0.446241   
3  6051000101         0.308717                 0.632135   
4  6093000200         0.795137                 0.343637   

   hospitality_access_pct  live_work_score_pct  shopping_access_pct  \
0                0.919383             0.883299             0.126099   
1                0.191843             0.68123

In [10]:
esg_data.dtypes
esg_data_numeric = esg_data.apply(pd.to_numeric, errors='ignore')
esg_data_numeric.dtypes

GEOID                         int64
jobs_access_pct             float64
education_diversity_pct     float64
hospitality_access_pct      float64
live_work_score_pct         float64
shopping_access_pct         float64
income_diversity_pct        float64
employment_diversity_pct    float64
income_harmony_pct          float64
edu_harmony_pct             float64
healthcare_access_pct       float64
geometry                     object
dtype: object

In [11]:
#Save to file
esg_data.to_file("esg.geojson", driver="GeoJSON")

In [ ]:
#combine data
#esg = gpd.read_file('./esg.geojson')

#combined_data = esg
#combined_data = combined_data.merge(svi_data, how='left', on='GEOID')
#combined_data = combined_data.merge(demographic_data, how='left', on='GEOID')

#combined_data

In [16]:
#Merge data
import json


with open('esg.geojson','r') as f:
    ct_geojson = json.load(f)

dict_svi=svi_data.set_index('GEOID')['SVI_total']

dict_total_pop=demographic_data.set_index('GEOID')['Total']
dict_two_or_more=demographic_data.set_index('GEOID')['Perc_two_or_more']
dict_white=demographic_data.set_index('GEOID')['Perc_white']
dict_black=demographic_data.set_index('GEOID')['Perc_black_afam']
dict_native=demographic_data.set_index('GEOID')['Perc_amindian_alnative']
dict_asian=demographic_data.set_index('GEOID')['Perc_asian']
dict_islander=demographic_data.set_index('GEOID')['Perc_hawaii_pi']
dict_other=demographic_data.set_index('GEOID')['Perc_other_race']
dict_u18=demographic_data.set_index('GEOID')['PercU18']


#Loop over GeoJSON features and add the new properties
for feat in ct_geojson['features']:
    try: 
        ct_id=feat['properties']['GEOID']
        
        ct_id=float(ct_id)
        
        svi=dict_svi[ct_id]
        feat['properties']['SVI_total']=str(svi)
        
        total_pop=dict_total_pop[ct_id]
        feat['properties']['Total_pop']=str(total_pop)
                                                   
        perc_two_or_more=dict_two_or_more[ct_id]
        feat['properties']['Perc_two_or_more']=str(perc_two_or_more)
        perc_white=dict_white[ct_id]
        feat['properties']['Perc_white']=str(perc_white)
        perc_black=dict_black[ct_id]
        feat['properties']['Perc_black_afam']=str(perc_black)
        perc_native=dict_native[ct_id]
        feat['properties']['Perc_amindian_alnative']=str(perc_native)
        perc_asian=dict_asian[ct_id]
        feat['properties']['Perc_asian']=str(perc_asian)
        perc_islander=dict_islander[ct_id]
        feat['properties']['Perc_hawaii_pi']=str(perc_islander)
        perc_other=dict_other[ct_id]
        feat['properties']['Perc_other_race']=str(perc_other)
        perc_u18=dict_u18[ct_id]
        feat['properties']['PercU18']=str(perc_u18)
        
        
    
    except KeyError:
        pass

#Write result to a new file
with open('new_combined_CAL.geojson', 'w') as f:
    json.dump(ct_geojson, f)

In [ ]:
#See result
#combined_data.head()

In [ ]:
#Save to file
#combined_data.to_file("combined_data.geojson", driver="GeoJSON")